In [1]:
import functools
import textwrap
from collections import OrderedDict


In [24]:
## QueryBuilder
class Query:
    keywords = [
        "SELECT",
        "CREATE TABLE IF NOT EXISTS",
        "INSERT INTO",
        "VALUES",
        "WHERE"
    ]
    separator = ','
    logic_separator = ' AND'
    _indent = functools.partial(textwrap.indent, prefix='')
    def __init__(self):
        self.data = OrderedDict([[k, []] for k in self.keywords])
    def clean_up(self, string : str):
        return textwrap.dedent(string.rstrip()).strip()
    def add(self, keyword, fields):
        target = self.data[keyword]
        if len(fields) == 0:
            target.append(self.clean_up('*'))
        else:
            for field in fields:
                target.append(self.clean_up(field))
        return self
    
    def __getattr__(self, name):
        if not name.isupper():
            return getattr(super(), name)
        return functools.partial(self.add, name.replace('_', ' '))

    def values_keyword(self, keyword, values):
        for i, value in enumerate(values, 1):
            last = i == len(values)
            yield self._indent(value)
            if not last:
                if keyword=="INSERT INTO":
                    yield " "
                    continue
                if keyword!='WHERE':
                    yield self.separator
                else: yield self.logic_separator
            yield ' '
    def keywords_put(self):
        for keyword, value in self.data.items():
            if not value:
                continue
            yield keyword + " "
            yield from self.values_keyword(keyword, value)
    def __str__(self):
        return ''.join(self.keywords_put())

In [31]:
fields = ('id', 'class')
string = '(' + ', '.join(fields) + ')'
string

'(id, class)'

In [34]:
def parse_values(values):
    return '(' + ', '.join(values) + ')'

In [37]:
def insert(table, fields, values):
    return str(Query().INSERT_INTO([table, parse_values(fields)]).VALUES([parse_values(values)]))

In [38]:
insert("aaa", ('id', 'class'), ('212', 'abba'))

'INSERT INTO aaa (id, class) VALUES (212, abba) '

In [27]:
str(Query().INSERT_INTO(['table', '(id, class)']).VALUES(['(4, asd)']))

'INSERT INTO table (id, class) VALUES (4, asd) '

In [40]:
import pandas as pd

pandas = pd.read_excel("BirdsFinal.xlsx")
pandas['id'] = pandas.index + 1

,bird_en,bird_ru,desc_en,desc_ru,place_ru,place_en,size_ru,size_en,id
0,BANDED BROADBILL,Яванский рогоклюв,The banded broadbill (Eurylaimus javanicus) is...,Яванский рогоклюв (лат. Eurylaimus javanicus) ...,Вид распространён в Юго-Восточной Азии. Встреч...,Mainland Southeast Asia and the Greater Sunda ...,"Птица достигает длины 21,5—23 см. Тело крепкое...","A striking, large-bodied bird with a length of...",1
1,AUSTRAL CANASTERO,Южный канастеро,The austral canastero (Asthenes anthoides) is ...,Австралийский канастеро ( Asthenes anthoides )...,NaN,NaN,NaN,NaN,2
2,ARARIPE MANAKIN,Шлемоносный манакин-арарипе,The Araripe manakin (Antilophia bokermanni) is...,Шлемоносный манакин-арарипе (лат. Antilophia b...,NaN,NaN,NaN,NaN,3
3,BAIKAL TEAL,Чирок-клоктун,"The Baikal teal (Sibirionetta formosa), also c...","Чирок-клоктун, или клоктун (лат. Sibirionetta ...",NaN,NaN,NaN,NaN,4
4,ALBERTS TOWHEE,Чернолицый тохи,"Large, plain, long-tailed sparrow. Drab grayis...","Большой, простой, длиннохвостый воробей. Весь ...",NaN,NaN,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...
505,WRENTIT,Американская тимелия,The wrentit (Chamaea fasciata) is a small bird...,"Америка́нская тиме́лия, или крапи́вниковая син...",NaN,NaN,NaN,NaN,506
506,YELLOW BELLIED FLOWERPECKER,Желтобрюхий цветоед,The yellow-bellied flowerpecker (Dicaeum melan...,Желтобрюхий цветоед (лат. Dicaeum melanoxanthu...,NaN,NaN,NaN,NaN,507
507,YELLOW CACIQUE,Желтопоясничный чёрный кассик,The yellow-rumped cacique (Cacicus cela) is a ...,Желтопоясничный чёрный кассик (лат. Cacicus ce...,NaN,NaN,NaN,NaN,508
508,YELLOW HEADED BLACKBIRD,Желтоголовый трупиал,The yellow-headed blackbird (Xanthocephalus xa...,Желтоголовый трупиал (лат. Xanthocephalus xant...,NaN,NaN,NaN,NaN,509
